```{=typst}
#set text(
  font: ("Times New Roman", "LXGW WenKai"),
  size: 11pt,
)

= 问题

#h(2em) 某公司将3种不同含硫量的液体原料（分别记为甲、乙、丙）混合生产两种产品（分别记为 A，B）。按照生产工艺的要求，原料甲、乙必须首先导入混合池中混合，混合后的液体再分别与原料丙混合生产 A，B。已知原料甲、乙、丙的含硫量分别是 3%，1%，2%，进货价格分别为 6 千元/t，16 千元/t，10 千元/t；产品 A，B 的含硫量分别不能超过 2.5%，1.5%，售价分别为 9 千元/t，15 千元/t。根据市场信息，原料甲、乙、丙的供应量都不能超过 500t；产品 A，B 的最大市场需求量分别为 100t，200t。

#h(2em) (1) 应如何安排生产？

#h(2em) (2) 如果产品 A 的最大市场需求量增长为 600t，应如何安排生产？

#h(2em) (3) 如果乙的进货价格下降为 13 千元/t，应如何安排生产？分别对 (1)、(2) 两种情况进行讨论。

= 思路

#h(2em) 设原料甲、乙分别用 $x_1$、$x_2$ 吨，分别取混合后的液体 $x_3$、$x_4$ 吨再加入原料丙 $x_5$、$x_6$ 吨生产产品 A、B。则有：

$
x_1 + x_2 = x_3 + x_4 ("质量守恒")
$

#h(2em) 限制条件：

$
x_1 <= 500 \
x_2 <= 500 \
x_5 + x_6 <= 500 \
x_3 + x_5 <= 100 \
x_4 + x_6 <= 200 \
x_1, x_2, x_3, x_4, x_5, x_6 >= 0
$

#h(2em) 甲乙混合后的液体含硫量 $alpha$ 可表示为（单位：%）：

$
alpha = (3x_1 + x_2) / (x_1 + x_2)
$

#h(2em) 含硫量限制条件：

$
alpha * x_3 + 2 * x_5 <= 2.5 * (x_3 + x_5) \
alpha * x_4 + 2 * x_6 <= 1.5 * (x_4 + x_6)
$

#h(2em) 公司净利润 $f$ 可表示为（单位：千元）：

$
f &= 9(x_3 + x_5) + 15(x_4 + x_6) - 6x_1 - 16x_2 - 10(x_5 + x_6) \
&= - 6x_1 - 16x_2 + 9x_3 + 15x_4 - x_5 + 5x_6
$

#h(2em) 即非线性规划求 $f$ 的最大值。

#h(2em) 调用 scipy.optimize 的 minimize 函数求解。

#h(2em) 下面是代码实现：

```


In [ ]:
import numpy as np
import scipy.optimize as opt

In [ ]:
N = 6


def f(X):
  coff = np.array([-6, -16, 9, 15, -1, 5])
  # 取反求最大值
  return -np.dot(coff, X)


def cons1(X):
  c1 = ((0.03 * X[0] + 0.01 * X[1]) * X[2] / (X[0] + X[1])) + 0.02 * X[4] - 0.025 * (X[2] + X[4])

  c2 = ((0.03 * X[0] + 0.01 * X[1]) * X[3] / (X[0] + X[1])) + 0.02 * X[5] - 0.015 * (X[3] + X[5])
  return np.array([c1, c2])


A0 = np.array(
  [
    [1, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 1],
    [0, 0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0, 1],
  ]
)
b0 = np.array([500, 500, 500, 100, 200])

nlc0 = opt.NonlinearConstraint(lambda X: np.dot(A0, X), np.zeros(5), b0)
nlc1 = opt.NonlinearConstraint(cons1, -np.inf * np.ones(2), np.zeros(2))
nlc2 = opt.NonlinearConstraint(lambda X: np.dot(np.array([1, 1, -1, -1, 0, 0]), X), 0, 0)
nlc3 = opt.NonlinearConstraint(lambda X: X, np.zeros(N), np.ones(N) * np.inf)

X0 = np.array([0, 100, 0, 100, 0, 100])
res = opt.minimize(f, X0, constraints=[nlc0, nlc1, nlc2, nlc3])

print(res.x)
print(-res.fun)

（1）

公司进货为甲 0t，乙 100t，丙 100t，全部用于生产 200t 产品 B，获利 400 千元。


In [ ]:
b1 = np.array([500, 500, 500, 600, 200])

nlc4 = opt.NonlinearConstraint(lambda X: np.dot(A0, X), np.zeros(5), b1)

X0 = np.array([300, 0, 300, 0, 300, 0])
res = opt.minimize(f, X0, constraints=[nlc4, nlc1, nlc2, nlc3])

print(res.x)
print(-res.fun)

（2）

公司进货为甲 300t，乙 0t，丙 300t，全部用于生产 600t 产品 A，获利 600 千元。


In [ ]:
def f1(X):
  coff = np.array([-6, -13, 9, 15, -1, 5])
  # 取反求最大值
  return -np.dot(coff, X)


X0 = np.array([50, 150, 0, 200, 0, 0])
res = opt.minimize(f1, X0, constraints=[nlc0, nlc1, nlc2, nlc3])

print(res.x)
print(-res.fun)

X0 = np.array([50, 150, 0, 200, 0, 0])
res = opt.minimize(f1, X0, constraints=[nlc4, nlc1, nlc2, nlc3])

print(res.x)
print(-res.fun)

（3）

- （1）公司进货为甲 50t，乙 150t，丙 0t，全部用于生产 200t 产品 B，获利 750 千元。
- （2）公司进货为甲 50t，乙 150t，丙 0t，全部用于生产 200t 产品 B，获利 750 千元。


```{=typst}

```
